# CheckerBoard Calibration

In [ ]:
import cv2
import os
import numpy as np
import glob

# 체커보드 내부 코너 수 (정확히 맞춰야 함)
CHECKERBOARD = (9, 6)  # 예: 10x7 체스판이면 (9,6)

# 실제 사각형 한 칸의 크기 (단위: m, 예: 0.025m = 25mm)
square_size = 0.0255

# 월드 좌표계에서 3D 좌표
objp = np.zeros((CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2) * square_size

objpoints = []  # 3D 점들
imgpoints = []  # 2D 점들

base_dir = os.getcwd()
images = glob.glob(os.path.join(base_dir, 'calibration', '*.jpg'))
print("이미지 개수:", len(images))

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, None)

    if ret:
        # 코너 정제
        corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)

        objpoints.append(objp)
        imgpoints.append(corners2)

        # 코너 시각화 (선택)
        cv2.drawChessboardCorners(img, CHECKERBOARD, corners, ret)
        cv2.imshow('img', img)
        cv2.waitKey(100)
    else:
        print(f"체스판 감지 실패: {fname}")

cv2.destroyAllWindows()

# ✅ gray 정의가 반복문에서 되고 있음
# 마지막 gray 사용 → 이미지 해상도용
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
    objpoints, imgpoints, gray.shape[::-1], None, None)

print("Camera Matrix:\n", mtx)
print("Distortion Coefficients:\n", dist)
print("RMS re-projection error:", ret) ## 결과 품질 확인

# # 저장 (선택)
# np.savez("/home/choigh/WS/Test_Tools_code/calib_data.npz", mtx=mtx, dist=dist, rvecs=rvecs, tvecs=tvecs)


이미지 개수: 34
Camera Matrix:
 [[712.45526183   0.         314.91146599]
 [  0.         710.75098385 245.34896623]
 [  0.           0.           1.        ]]
Distortion Coefficients:
 [[ 2.94641622e-02  7.36845855e-01 -1.18703311e-03 -4.73634944e-04
  -2.06935893e+00]]
RMS re-projection error: 0.31588105123422744


# Camera Matrix & Distortion Coeff

In [3]:
import numpy as np

base_dir = os.getcwd()
data = np.load(os.path.join(base_dir, 'calib_data.npz'))
mtx = data['mtx']
dist = data['dist']

print("fx, fy =", mtx[0, 0], mtx[1, 1])
print("cx, cy =", mtx[0, 2], mtx[1, 2])
print("distortion =", dist)


fx, fy = 724.785668389083 720.7425294440777
cx, cy = 322.1334139745953 235.07954611284364
distortion = [[ 1.55359511e-02  1.11160274e+00  6.03520333e-04  3.18295542e-03
  -3.77598056e+00]]
